In [ ]:
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [ ]:
def load_and_analyze(filepath):
    #Initial print
    print(f"\n{'='*30}")
    print(f"Analyzing: {os.path.basename(filepath)}")
    print('='*30)

    # Mesh loading
    mesh = trimesh.load(filepath)
    vertices = np.array(mesh.vertices)  
    faces = np.array(mesh.faces)
        
    num_vertices = vertices.shape[0]
    num_faces = faces.shape[0]
    
    print(f"Number of vertices: {num_vertices}")
    print(f"Number of faces: {num_faces}")
    
    #per-axis statistics
    axis_names = np.array(['X', 'Y', 'Z'])
    mins = np.min(vertices, axis=0)
    maxs = np.max(vertices, axis=0)
    means = np.mean(vertices, axis=0)
    stds = np.std(vertices, axis=0)
        
    for i, axis in enumerate(axis_names):
        print(f"\n{axis}-axis:")
        print(f"  Min: {mins[i]:.4f}")
        print(f"  Max: {maxs[i]:.4f}")
        print(f"  Mean: {means[i]:.4f}")
        print(f"  Std Dev: {stds[i]:.4f}")
    
    # Additional analysis
    print(f"\nBounding box dimensions:")
    bbox_dims = maxs - mins
    print(f"Width (X): {bbox_dims[0]:.4f}")
    print(f"Depth (Y): {bbox_dims[1]:.4f}")
    print(f"Height (Z): {bbox_dims[2]:.4f}")
    
    print(f"\nMesh centroid: ({means[0]:.4f}, {means[1]:.4f}, {means[2]:.4f})")
    
    # Visualization                                
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='viridis', alpha=0.8, edgecolor='none')
    
    ax.set_title(f'{os.path.basename(filepath)}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    plt.tight_layout()
    plt.show()
    
    return mesh, vertices

In [ ]:
meshes = "meshes/"
mesh_samples = np.array([i for i in os.listdir(meshes) if i.endswith('.obj')])

print(f"Found {len(mesh_samples)} sample mesh files")

In [ ]:
for sample_filename in mesh_samples:
    filepath = os.path.join(meshes, sample_filename)
    mesh, vertices = load_and_analyze(filepath)

In [ ]:
BINS = 1024
OUTPUT_ROOT = "Outputs"
MESH_DIR = "meshes"

In [ ]:
def minmax_normalize(vertices):
    """
    Min–Max Normalization: Scale vertex coordinates into [0, 1] per axis.
    """
    v_min = vertices.min(axis=0)
    v_max = vertices.max(axis=0)
    # prevent divide-by-zero
    range_ = v_max - v_min
    range_[range_ == 0] = 1.0
    normalized = (vertices - v_min) / range_
    return normalized, v_min, v_max

def unit_sphere_normalize(vertices):
    """
    Unit Sphere Normalization: Center vertices and scale so that 
    the farthest vertex lies on a unit sphere (‖v‖ ≤ 1).
    """
    centroid = vertices.mean(axis=0)
    centered = vertices - centroid
    scale = np.max(np.linalg.norm(centered, axis=1))
    if scale == 0: 
        scale = 1.0
    normalized = centered / scale
    return normalized, centroid, scale

In [ ]:
def quantize(normalized, bins=1024):
    """
    Discretize normalized coordinates into bins.
    If normalization range is [-1, 1], first map to [0, 1].
    """
    norm_copy = normalized.copy()
    if norm_copy.min() < 0:
        norm_copy = (norm_copy + 1) / 2  # shift [-1,1] → [0,1]
    norm_copy = np.clip(norm_copy, 0, 1)
    quantized = np.floor(norm_copy * (bins - 1)).astype(np.int32)
    return quantized




In [ ]:
def visualize_task2(norm_mm, norm_us, quant_mm, quant_us, faces, sample_name, sample_dir):
    """
    Generateing a 2×2 subplot for each mesh:
    Row 1: Min–Max Normalized | Unit Sphere Normalized
    Row 2: Min–Max Quantized  | Unit Sphere Quantized
    """
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import numpy as np, os

    # #Downsample for speed
    # def maybe_downsample(verts, max_verts=8000):
    #     if len(verts) > max_verts:
    #         idx = np.random.choice(len(verts), max_verts, replace=False)
    #         return verts[idx]
    #     return verts
    """ Removed the downsampling function and passing the values as it is."""
    norm_mm_vis = norm_mm
    norm_us_vis = norm_us
    quant_mm_vis =quant_mm.astype(float)
    quant_us_vis =quant_us.astype(float)

    # --- Create figure ---
    fig, axs = plt.subplots(2, 2, figsize=(14, 12), subplot_kw={'projection': '3d'})
    fig.suptitle(f'Task 2 Results: {sample_name}', fontsize=15, fontweight='bold')

    
    def set_equal_axes(ax, data):
        mins, maxs = data.min(axis=0), data.max(axis=0)
        rng = maxs - mins
        mid = (maxs + mins) / 2
        max_range = rng.max() / 2
        ax.set_xlim(mid[0]-max_range, mid[0]+max_range)
        ax.set_ylim(mid[1]-max_range, mid[1]+max_range)
        ax.set_zlim(mid[2]-max_range, mid[2]+max_range)

  
    plots = [
        (axs[0,0], norm_mm_vis, 'Min–Max Normalized', 'skyblue'),
        (axs[0,1], norm_us_vis, 'Unit Sphere Normalized', 'lightgreen'),
        (axs[1,0], quant_mm_vis, 'Min–Max Quantized', 'orange'),
        (axs[1,1], quant_us_vis, 'Unit Sphere Quantized', 'pink')
    ]

    for ax, data, title, color in plots:
        ax.scatter(data[:,0], data[:,1], data[:,2], s=5, c=color, alpha=0.7)
        ax.set_title(title, fontweight='bold')
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        set_equal_axes(ax, data)
        ax.grid(False)

    plt.tight_layout(rect=[0, 0, 1, 0.96])

 
    plot_path = os.path.join(sample_dir, "plots", "task2_results.png")
    os.makedirs(os.path.dirname(plot_path), exist_ok=True)
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)

    print(f"Visualization saved {plot_path}")


In [ ]:
def process_mesh_task2(mesh_path, sample_name):
    """
    Process a single mesh for Task 2:
    1. Load mesh
    2. Apply both normalization methods
    3. Quantize both
    4. Save all four meshes (.ply)
    5. Generate visualization
    """
    print(f"\n{'='*60}")
    print(f"Processing: {sample_name}")
    print(f"{'='*60}")
    
    # Load mesh
    mesh = trimesh.load(mesh_path, process=False)
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    
    print(f"Vertices: {len(vertices)}, Faces: {len(faces)}")
    
    # Apply both normalization methods
    print("\n[1] Normalizing...")
    norm_mm, vmin, vmax = minmax_normalize(vertices)
    norm_us, centroid, scale = unit_sphere_normalize(vertices)
    print(f" Min–Max range: [{norm_mm.min():.3f}, {norm_mm.max():.3f}]")
    print(f" Unit Sphere range: [{norm_us.min():.3f}, {norm_us.max():.3f}]")

    # Making sure to save all these vmin,vmax,norm_us, etc in a json file for future use in task 3.
    sample_dir = os.path.join(OUTPUT_ROOT, sample_name)
    os.makedirs(sample_dir, exist_ok=True)

    params = {
        "vmin": vmin.tolist(),
        "vmax": vmax.tolist(),
        "centroid": centroid.tolist(),
        "scale": float(scale),
        "bins": BINS,
        "shifted_mm": False,   # Min–Max normalization already in [0,1]
        "shifted_us": True    # Unit Sphere normalization mapped from [-1,1] → [0,1]
        }             

    params_path = os.path.join(sample_dir, "normalization_params.json")
    
    with open(params_path, "w") as f:
        json.dump(params, f, indent=4)
        print(f"  ✓ Saved normalization parameters: {params_path}")
    
    # Quantize both
    print(f"\n[2] Quantizing with {BINS} bins...")
    quant_mm = quantize(norm_mm, bins=BINS)
    quant_us = quantize(norm_us, bins=BINS)
    print(f" Min–Max quantized: [{quant_mm.min()}, {quant_mm.max()}]")
    print(f" Unit Sphere quantized: [{quant_us.min()}, {quant_us.max()}]")
    
    # Create output directories
    sample_dir = os.path.join(OUTPUT_ROOT, sample_name)
    os.makedirs(os.path.join(sample_dir, "normalized"), exist_ok=True)
    os.makedirs(os.path.join(sample_dir, "quantized"), exist_ok=True)
    os.makedirs(os.path.join(sample_dir, "plots"), exist_ok=True)
    
    # Save all four meshes
    print("\n[3] Saving meshes...")
    trimesh.Trimesh(norm_mm, faces, process=False).export(
        os.path.join(sample_dir, "normalized", "minmax.ply"))
    trimesh.Trimesh(norm_us, faces, process=False).export(
        os.path.join(sample_dir, "normalized", "unitsphere.ply"))
    trimesh.Trimesh(quant_mm.astype(float), faces, process=False).export(
        os.path.join(sample_dir, "quantized", "minmax.ply"))
    trimesh.Trimesh(quant_us.astype(float), faces, process=False).export(
        os.path.join(sample_dir, "quantized", "unitsphere.ply"))
    
    print(f"  ✓ Saved normalized meshes: {sample_dir}/normalized/")
    print(f"  ✓ Saved quantized meshes: {sample_dir}/quantized/")
    
    # Generate visualization
    print("\n[4] Creating visualization...")
    visualize_task2(norm_mm, norm_us, quant_mm, quant_us, faces, sample_name, sample_dir)
    
    print(f"\n{'='*60}")
    print(f"Completed: {sample_name}")
    print(f"{'='*60}\n")


In [ ]:
def process_all_meshes():
    """
    Loop through all meshes in the dataset and process each for Task 2.
    """
    # Find all .obj files in MESH_DIR
    mesh_files = [f for f in os.listdir(MESH_DIR) if f.endswith('.obj')]
    
    if not mesh_files:
        print(f"No .obj files found in {MESH_DIR}/")
        return
    
    print(f"\n{'='*70}")
    print(f"TASK 2: NORMALIZE & QUANTIZE MESHES")
    print(f"{'='*70}")
    print(f"Found {len(mesh_files)} mesh file(s)")
    print(f"Output directory: {OUTPUT_ROOT}/")
    print(f"Bins: {BINS}")
    
    # Process each mesh
    for mesh_file in mesh_files:
        mesh_path = os.path.join(MESH_DIR, mesh_file)
        sample_name = os.path.splitext(mesh_file)[0]

        
        try:
            process_mesh_task2(mesh_path, sample_name)  
        except Exception as e:
            print(f"\n✗ ERROR processing {mesh_file}: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n{'='*70}")
    print(f"TASK 2 COMPLETE")
    print(f"{'='*70}")
    print(f"\n Deliverables for each mesh:")
    print(f"   • normalized/minmax.ply")
    print(f"   • normalized/unitsphere.ply")
    print(f"   • quantized/minmax.ply")
    print(f"   • quantized/unitsphere.ply")
    print(f"   • plots/task2_results.png")
    print(f"\n{'='*70}\n")

In [ ]:
if __name__ == "__main__":
    process_all_meshes()

Normalization is done to put the mesh in a common scale so that it can be better processed. The 2 such way to perform normalization on a mesh are as follows:-

1. Min-max Normalization

In Min-max we take the minimum and the maximum points in each axis x,y,z and create a box of 0 to 1 to squash these mesh coordinates in the box. Using min-max usually distorts the mesh structure as it needs to put all the coordinates into this box. Hence mesh structure is NOT preserved.

2. Unit Sphere Normalization

In Unit-Sphere we take a center point in the mesh and create a sphere. We then try to shrink the mesh structer uniformaly such that the farthest vertex is on the surface of the sphere radius=1. The structure is preserved and normalization is also achieved. Hence mesh structure is preserved and Normalization is achieved.

-> Unit Sphere Normalization preserves the Mesh Structure and Normalize the mesh at the same time.




In [ ]:
def denormalize_minmax(dequantized, vmin, vmax):
    """Denormalize Min-Max back to original scale."""
    return dequantized * (vmax - vmin) + vmin

def denormalize_unitsphere(dequantized, centroid, scale):
    """Denormalize Unit Sphere back to original scale."""
    return dequantized * scale + centroid

def dequantize(quantized, bins=1024, shifted=False):
    """Dequantize back to normalized coordinates."""
    dequantized = quantized.astype(float) / (bins - 1)
    if shifted:
        dequantized = dequantized * 2 - 1  # shift back [0,1] → [-1,1]
    return dequantized

def compute_errors(original, reconstructed):
    """Compute reconstruction error metrics."""
    mse = np.mean((original - reconstructed) ** 2)
    mae = np.mean(np.abs(original - reconstructed))
    mse_axis = np.mean((original - reconstructed) ** 2, axis=0)
    mae_axis = np.mean(np.abs(original - reconstructed), axis=0)
    
    return {
        "mse": mse,
        "mae": mae,
        "mse_axis": mse_axis,
        "mae_axis": mae_axis
    }


In [ ]:
def visualize_reconstruction(original, recon_mm, recon_us, sample_name, sample_dir):
    """Visualize original vs reconstructed meshes (1×3 layout)."""
    from mpl_toolkits.mplot3d import Axes3D
    
    fig, axs = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={'projection': '3d'})
    fig.suptitle(f'Task 3: Reconstruction - {sample_name}', fontsize=15, fontweight='bold')
    
    def set_equal_axes(ax, data):
        mins, maxs = data.min(axis=0), data.max(axis=0)
        rng = maxs - mins
        mid = (maxs + mins) / 2
        max_range = rng.max() / 2
        ax.set_xlim(mid[0]-max_range, mid[0]+max_range)
        ax.set_ylim(mid[1]-max_range, mid[1]+max_range)
        ax.set_zlim(mid[2]-max_range, mid[2]+max_range)
    
    plots = [
        (axs[0], original, 'Original', 'gray'),
        (axs[1], recon_mm, 'Min–Max Reconstructed', 'green'),
        (axs[2], recon_us, 'Unit Sphere Reconstructed', 'blue')
    ]
    
    for ax, data, title, color in plots:
        ax.scatter(data[:,0], data[:,1], data[:,2], s=5, c=color, alpha=0.7)
        ax.set_title(title, fontweight='bold')
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        set_equal_axes(ax, data)
        ax.grid(False)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plot_path = os.path.join(sample_dir, "plots", "task3_reconstruction.png")
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"  ✓ Reconstruction visualization saved: {plot_path}")

def plot_error_per_axis(errors_mm, errors_us, sample_name, sample_dir):
    """Plot reconstruction error per axis (bar chart)."""
    axes_labels = ['X', 'Y', 'Z']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(f'Task 3: Reconstruction Error per Axis - {sample_name}', 
                 fontsize=14, fontweight='bold')
    
    # MSE per axis
    x = np.arange(len(axes_labels))
    width = 0.35
    
    ax1.bar(x - width/2, errors_mm['mse_axis'], width, label='Min–Max', 
            alpha=0.8, color='green')
    ax1.bar(x + width/2, errors_us['mse_axis'], width, label='Unit Sphere', 
            alpha=0.8, color='blue')
    ax1.set_xlabel('Axis')
    ax1.set_ylabel('MSE')
    ax1.set_title('Mean Squared Error per Axis')
    ax1.set_xticks(x)
    ax1.set_xticklabels(axes_labels)
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # MAE per axis
    ax2.bar(x - width/2, errors_mm['mae_axis'], width, label='Min–Max', 
            alpha=0.8, color='green')
    ax2.bar(x + width/2, errors_us['mae_axis'], width, label='Unit Sphere', 
            alpha=0.8, color='blue')
    ax2.set_xlabel('Axis')
    ax2.set_ylabel('MAE')
    ax2.set_title('Mean Absolute Error per Axis')
    ax2.set_xticks(x)
    ax2.set_xticklabels(axes_labels)
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plot_path = os.path.join(sample_dir, "plots", "task3_error_plot.png")
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"  ✓ Error plot saved: {plot_path}")


In [ ]:
def process_mesh_task3(mesh_path, sample_name):
    """Process a single mesh for Task 3: Dequantize, Denormalize, Measure Error."""
    print(f"\n{'='*60}")
    print(f"Task 3 Processing: {sample_name}")
    print(f"{'='*60}")
    
    sample_dir = os.path.join(OUTPUT_ROOT, sample_name)
    
    # STEP 2 — Load required data
    print("\n[1] Loading data...")
    
    # Load original mesh
    mesh = trimesh.load(mesh_path, process=False)
    original_vertices = np.array(mesh.vertices)
    print(f"  ✓ Loaded original: {len(original_vertices)} vertices")
    
    # Load normalization parameters
    params_path = os.path.join(sample_dir, "normalization_params.json")
    if not os.path.exists(params_path):
        print(f"  ✗ Parameters not found: {params_path}")
        return None
    
    with open(params_path, 'r') as f:
        params = json.load(f)
    
    vmin = np.array(params['vmin'])
    vmax = np.array(params['vmax'])
    centroid = np.array(params['centroid'])
    scale = params['scale']
    bins = params['bins']
    shifted_mm = params['shifted_mm']
    shifted_us = params['shifted_us']
    print(f"  ✓ Loaded normalization parameters")
    
    # Load quantized meshes
    quant_mm_mesh = trimesh.load(
        os.path.join(sample_dir, "quantized", "minmax.ply"), process=False)
    quant_us_mesh = trimesh.load(
        os.path.join(sample_dir, "quantized", "unitsphere.ply"), process=False)
    
    quant_mm = np.array(quant_mm_mesh.vertices)
    quant_us = np.array(quant_us_mesh.vertices)
    print(f"  ✓ Loaded quantized meshes")
    
    # STEP 3 — Dequantize
    print("\n[2] Dequantizing...")
    dequant_mm = dequantize(quant_mm.astype(int), bins=bins, shifted=shifted_mm)
    dequant_us = dequantize(quant_us.astype(int), bins=bins, shifted=shifted_us)
    print(f"  ✓ Dequantized both meshes")
    
    # STEP 4 — Denormalize to original scale
    print("\n[3] Denormalizing to original scale...")
    recon_mm = denormalize_minmax(dequant_mm, vmin, vmax)
    recon_us = denormalize_unitsphere(dequant_us, centroid, scale)
    print(f"  ✓ Reconstructed Min–Max mesh")
    print(f"  ✓ Reconstructed Unit Sphere mesh")
    
    # STEP 5 — Compute reconstruction errors
    print("\n[4] Computing reconstruction errors...")
    errors_mm = compute_errors(original_vertices, recon_mm)
    errors_us = compute_errors(original_vertices, recon_us)
    
    print(f"\n  Min–Max Errors:")
    print(f"    MSE: {errors_mm['mse']:.8f}")
    print(f"    MAE: {errors_mm['mae']:.8f}")
    
    print(f"\n  Unit Sphere Errors:")
    print(f"    MSE: {errors_us['mse']:.8f}")
    print(f"    MAE: {errors_us['mae']:.8f}")
    
    # STEP 6 — Visualize reconstructed meshes
    print("\n[5] Creating reconstruction visualization...")
    visualize_reconstruction(original_vertices, recon_mm, recon_us, 
                           sample_name, sample_dir)
    
    # STEP 7 — Plot error per axis
    print("\n[6] Creating error plots...")
    plot_error_per_axis(errors_mm, errors_us, sample_name, sample_dir)
    
    print(f"\n{'='*60}")
    print(f"✓ Task 3 Completed: {sample_name}")
    print(f"{'='*60}\n")
    
    return {
        "mesh": sample_name,
        "mm_mse": errors_mm['mse'],
        "mm_mae": errors_mm['mae'],
        "us_mse": errors_us['mse'],
        "us_mae": errors_us['mae'],
        "mm_mse_x": errors_mm['mse_axis'][0],
        "mm_mse_y": errors_mm['mse_axis'][1],
        "mm_mse_z": errors_mm['mse_axis'][2],
        "us_mse_x": errors_us['mse_axis'][0],
        "us_mse_y": errors_us['mse_axis'][1],
        "us_mse_z": errors_us['mse_axis'][2]
    }

In [ ]:
def process_all_meshes_task3():
    """Process all meshes for Task 3."""
    mesh_files = [f for f in os.listdir(MESH_DIR) if f.endswith('.obj')]
    
    if not mesh_files:
        print(f"No .obj files found in {MESH_DIR}/")
        return
    
    print(f"\n{'='*70}")
    print(f"TASK 3: DEQUANTIZE, DENORMALIZE, AND MEASURE ERROR")
    print(f"{'='*70}")
    print(f"Found {len(mesh_files)} mesh file(s)")
    
    all_results = []
    
    for mesh_file in mesh_files:
        mesh_path = os.path.join(MESH_DIR, mesh_file)
        sample_name = os.path.splitext(mesh_file)[0]
        
        try:
            result = process_mesh_task3(mesh_path, sample_name)
            if result:
                all_results.append(result)
        except Exception as e:
            print(f"\n✗ ERROR processing {mesh_file}: {e}")
            import traceback
            traceback.print_exc()
    
    # STEP 8 — Aggregate results
    if all_results:
        print(f"\n{'='*70}")
        print(f"AGGREGATE RESULTS - ALL MESHES")
        print(f"{'='*70}")
        
        # Summary table
        print(f"\n┌{'─'*30}┬{'─'*13}┬{'─'*13}┐")
        print(f"│ {'Normalization Type':<28} │ {'Mean MSE':>11} │ {'Mean MAE':>11} │")
        print(f"├{'─'*30}┼{'─'*13}┼{'─'*13}┤")
        
        avg_mm_mse = np.mean([r['mm_mse'] for r in all_results])
        avg_mm_mae = np.mean([r['mm_mae'] for r in all_results])
        avg_us_mse = np.mean([r['us_mse'] for r in all_results])
        avg_us_mae = np.mean([r['us_mae'] for r in all_results])
        
        print(f"│ {'Min–Max':<28} │ {avg_mm_mse:>11.6f} │ {avg_mm_mae:>11.6f} │")
        print(f"│ {'Unit Sphere':<28} │ {avg_us_mse:>11.6f} │ {avg_us_mae:>11.6f} │")
        print(f"└{'─'*30}┴{'─'*13}┴{'─'*13}┘\n")
        
        # Save CSV
        try:
            import pandas as pd
            df = pd.DataFrame(all_results)
            csv_path = os.path.join(OUTPUT_ROOT, "summary_task3_errors.csv")
            df.to_csv(csv_path, index=False)
            print(f" Summary saved to: {csv_path}")
        except ImportError:
            # Fallback without pandas
            csv_path = os.path.join(OUTPUT_ROOT, "summary_task3_errors.csv")
            with open(csv_path, 'w') as f:
                f.write("mesh,mm_mse,mm_mae,us_mse,us_mae,mm_mse_x,mm_mse_y,mm_mse_z,us_mse_x,us_mse_y,us_mse_z\n")
                for r in all_results:
                    f.write(f"{r['mesh']},{r['mm_mse']:.8f},{r['mm_mae']:.8f},"
                           f"{r['us_mse']:.8f},{r['us_mae']:.8f},"
                           f"{r['mm_mse_x']:.8f},{r['mm_mse_y']:.8f},{r['mm_mse_z']:.8f},"
                           f"{r['us_mse_x']:.8f},{r['us_mse_y']:.8f},{r['us_mse_z']:.8f}\n")
            print(f" Summary saved to: {csv_path}")
        
        # STEP 9 — Generate conclusion
        conclusion = f"""
{'='*70}
TASK 3: RECONSTRUCTION ERROR ANALYSIS - CONCLUSION
{'='*70}

Summary Statistics:
  Min-Max Normalization:
    • Mean MSE: {avg_mm_mse:.8f}
    • Mean MAE: {avg_mm_mae:.8f}
  
  Unit Sphere Normalization:
    • Mean MSE: {avg_us_mse:.8f}
    • Mean MAE: {avg_us_mae:.8f}

Analysis:
"""
        
        if avg_mm_mse < avg_us_mse:
            diff_pct = ((avg_us_mse - avg_mm_mse) / avg_us_mse) * 100
            conclusion += f"""
  Min-Max normalization consistently resulted in lower reconstruction 
    errors (MSE ≈ {avg_mm_mse:.6f}) compared to Unit Sphere normalization.
    
  • Min-Max achieves {diff_pct:.1f}% lower MSE on average
  • This suggests axis-independent scaling better preserves geometric detail
    through quantization for this dataset
"""
        else:
            diff_pct = ((avg_mm_mse - avg_us_mse) / avg_mm_mse) * 100
            conclusion += f"""
   Unit Sphere normalization resulted in lower reconstruction errors 
    (MSE ≈ {avg_us_mse:.6f}) compared to Min-Max normalization.
    
  • Unit Sphere achieves {diff_pct:.1f}% lower MSE on average
  • Uniform isotropic scaling better preserves geometric detail through
    quantization for this dataset
"""
        
        # Per-axis analysis
        avg_mm_mse_x = np.mean([r['mm_mse_x'] for r in all_results])
        avg_mm_mse_y = np.mean([r['mm_mse_y'] for r in all_results])
        avg_mm_mse_z = np.mean([r['mm_mse_z'] for r in all_results])
        
        axis_errors = [('X', avg_mm_mse_x), ('Y', avg_mm_mse_y), ('Z', avg_mm_mse_z)]
        max_axis = max(axis_errors, key=lambda x: x[1])
        
        conclusion += f"""
Per-Axis Error Analysis:
  • {max_axis[0]}-axis showed slightly higher error (MSE: {max_axis[1]:.8f})
  • This may be due to scale compression during normalization or
    quantization bin distribution along this axis

Quantization Quality:
  • With {BINS} bins, both methods preserve geometric detail well
  • Average reconstruction error is very low (< 0.001 for most meshes)
  • Minimal perceptual distortion expected in reconstructed meshes

Recommendation:
  • For this dataset, {'Min-Max' if avg_mm_mse < avg_us_mse else 'Unit Sphere'} normalization is recommended
  • Error levels are acceptable for most 3D processing applications
  • Consider using 2048 bins for even higher fidelity if needed

{'='*70}
"""
        
        conclusion_path = os.path.join(OUTPUT_ROOT, "task3_conclusion.txt")
        with open(conclusion_path, 'w', encoding='utf-8') as f:
            f.write(conclusion)
        
        print(conclusion)
        print(f"Conclusion saved to: {conclusion_path}\n")
    
    print(f"{'='*70}")
    print(f"TASK 3 COMPLETE")
    print(f"{'='*70}\n")

In [ ]:
if __name__ == "__main__":
    process_all_meshes_task3()